<a href="https://colab.research.google.com/github/jedhouas/Digraph/blob/master/train_ppi_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# for CPU 
!pip  install dgl

#For GPU 
#!pip install dgl-cu101

     |████████████████████████████████| 2.4MB 2.8MB/s 


In [0]:
import argparse
from os import path
import numpy as np
import torch
import torch.nn.functional as F
from dgl import batch
from dgl.data.ppi import LegacyPPIDataset
from dgl.nn.pytorch import GraphConv 
from sklearn.metrics import f1_score
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from dgl.nn.pytorch import GATConv 

In [0]:
from dgl.nn.pytorch import GATConv 

In [0]:
class parser(object):
  def __init__(self,mode='train',gpu=0,epochs=250,batch_size=2,num_heads=8,num_out_heads=1): 
    self.mode=mode
    self.gpu=gpu 
    self.epochs=epochs
    self.batch_size=batch_size
    self.num_out_heads=num_out_heads
    self.num_heads=num_heads

In [0]:
MODEL_STATE_FILE = "model_state.pth"

In [0]:
from dgl.nn.pytorch import  GATConv

class GAT(nn.Module):
    def __init__(self,g,n_layers,input_size,hidden_size,output_size,heads,nonlinearity,feat_drop=0,attn_drop=0.1,negative_slope=0.2,residual=False):
        super(GAT, self).__init__()
        self.g = g
        self.n_layers = n_layers
        self.layers = nn.ModuleList()
        self.activation = nonlinearity
        # input projection 
        self.layers.append(GATConv(
            input_size, hidden_size, heads[0],
            feat_drop, attn_drop, negative_slope, False, self.activation))
        # hidden layers with Residual
        for l in range(1, n_layers):
            # due to multi-head, the in_dim = num_hidden * num_heads
            self.layers.append(GATConv(
                hidden_size * heads[l-1], hidden_size, heads[l],
                feat_drop, attn_drop, negative_slope, residual, self.activation))
        # output projection, with Residual
        self.layers.append(GATConv(
            hidden_size * heads[-2], output_size, heads[-1],
            feat_drop, attn_drop, negative_slope, residual, None))

    def forward(self, inputs):
        outputs = inputs
        for l in range(self.n_layers):
          #Concat heads output for intermediary layers 
          outputs = self.layers[l](self.g, outputs).flatten(1)
        # We use mean of heads for last Layer 
        logits = self.layers[-1](self.g, outputs).mean(1)
        return logits

In [0]:
class BasicGraphModel(nn.Module):
    def __init__(self, g, n_layers, input_size, hidden_size, output_size, nonlinearity):
        super().__init__()
        self.g = g
        self.layers = nn.ModuleList()
        self.layers.append(GATConv(input_size, hidden_size,num_heads=3,activation=nonlinearity))
        for i in range(n_layers - 1):
            self.layers.append(GATConv(hidden_size, hidden_size,num_heads=3,activation=nonlinearity))
        self.layers.append(GATConv(hidden_size, output_size,num_heads=3))

    def forward(self, inputs):
        outputs = inputs
        for i, layer in enumerate(self.layers):
            outputs = layer(self.g, outputs)
        return outputs

In [0]:
def train(model, loss_fcn, device, optimizer, train_dataloader, test_dataset,scheduler):
    for epoch in range(args.epochs):
        model.train() 
        losses = [] 
        for batch, data in enumerate(train_dataloader):
            subgraph, features, labels = data
            features = features.to(device)
            labels = labels.to(device)
            model.g = subgraph
            for layer in model.layers:
                layer.g = subgraph
            logits = model(features.float())
            loss = loss_fcn(logits, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        loss_data = np.array(losses).mean()
        print("Epoch {:05d} | Loss: {:.4f}".format(epoch + 1, loss_data))

        if epoch % 5 == 0:
            scores = []
            for batch, test_data in enumerate(test_dataset):
                subgraph, features, labels = test_data
                features = torch.tensor(features).to(device)
                labels = torch.tensor(labels).to(device)
                score, _ = evaluate(features.float(), model, subgraph, labels.float(), loss_fcn)
                scores.append(score)
            print("F1-Score: {:.4f} ".format(np.array(scores).mean()))
        scheduler.step()
def test(model, loss_fcn, device, test_dataloader):
    test_scores = []
    for batch, test_data in enumerate(test_dataloader):
        subgraph, features, labels = test_data
        features = features.to(device)
        labels = labels.to(device)
        test_scores.append(evaluate(features, model, subgraph, labels.float(), loss_fcn)[0])
    mean_scores = np.array(test_scores).mean()
    print("F1-Score: {:.4f}".format(np.array(test_scores).mean()))
    return mean_scores


def evaluate(features, model, subgraph, labels, loss_fcn):
    with torch.no_grad():
        model.eval()
        model.g = subgraph
        for layer in model.layers:
            layer.g = subgraph
        output = model(features.float())
        loss_data = loss_fcn(output, labels.float())
        predict = np.where(output.data.cpu().numpy() >= 0.5, 1, 0)
        score = f1_score(labels.data.cpu().numpy(), predict, average="micro")
        return score, loss_data.item()


def collate_fn(sample):
    graphs, features, labels = map(list, zip(*sample))
    graph = batch(graphs)
    features = torch.from_numpy(np.concatenate(features))
    labels = torch.from_numpy(np.concatenate(labels))
    return graph, features, labels


In [0]:
if torch.cuda.is_available() : 
  gpu=0 
else : 
  gpu=-1 
args=parser(gpu=gpu) 

# create the dataset 
train_dataset, test_dataset = LegacyPPIDataset(mode="train"), LegacyPPIDataset(mode="test")
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size, collate_fn=collate_fn)
n_features, n_classes = train_dataset.features.shape[1], train_dataset.labels.shape[1]

Extracting file to /root/.dgl/ppi
Loading G...
Loading G...


In [0]:
args.num_heads=10
args.num_out_heads=5
args.epochs=500


In [0]:
n_layers=4
heads = ([args.num_heads] * n_layers) + [args.num_out_heads] 
# create the model, loss function and optimizer 
device = torch.device("cpu" if args.gpu < 0 else "cuda:" + str(args.gpu))
model=GAT(g=train_dataset.graph,n_layers=n_layers,input_size=n_features,
          feat_drop=0.,attn_drop=0.1,hidden_size=256,output_size=n_classes,
          heads=heads,nonlinearity=F.elu,residual=True).to(device)
          
loss_fcn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
scheduler=lr_scheduler.StepLR(optimizer,step_size=300,gamma=0.1)

In [0]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

In [0]:
# train and test
if args.mode == "train":
    model.train()
    train(model , loss_fcn, device, optimizer, train_dataloader, test_dataset,scheduler)
torch.save(model.state_dict(), MODEL_STATE_FILE)
model.load_state_dict(torch.load(MODEL_STATE_FILE))
test(model, loss_fcn, device, test_dataloader)

KeyboardInterrupt: ignored

In [0]:
!python train_ppi.py --mode "test"

Loading G...
Loading G...
F1-Score: 0.3914


In [0]:
model.eval()
test(model, loss_fcn, device, test_dataloader)

F1-Score: 0.9923


0.9923130152235393

In [0]:
# create the model, loss function and optimizer
device = torch.device("cpu" if args.gpu < 0 else "cuda:" + str(args.gpu))
n_layers=2
model = BasicGraphModel(g=train_dataset.graph, n_layers=n_layers, input_size=n_features,
                        hidden_size=256, output_size=n_classes, nonlinearity=F.elu).to(device)
loss_fcn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [0]:
from graphviz import Digraph

def make_dot(var, params):
    """ Produces Graphviz representation of PyTorch autograd graph
    
    Blue nodes are the Variables that require grad, orange are Tensors
    saved for backward in torch.autograd.Function
    
    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    param_map = {id(v): k for k, v in params.items()}
    print(param_map)
    
    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()
    
    def size_to_str(size):
        return '('+(', ').join(['%d'% v for v in size])+')'

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                node_name = '%s\n %s' % (param_map.get(id(u)), size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    add_nodes(var.grad_fn)
    return dot

a,b,c=next(iter(train_dataloader))
model.g = a
for layer in model.layers:
    layer.g = a
y = model(b.float().to(device))
g = make_dot(y, model.state_dict())
g.view()

{140093222727040: 'layers.0.attn_l', 140093222710224: 'layers.0.attn_r', 140093222707632: 'layers.0.fc.weight', 140093222752328: 'layers.1.attn_l', 140093222752400: 'layers.1.attn_r', 140093222752544: 'layers.1.fc.weight', 140093222753264: 'layers.1.res_fc.weight', 140093222753408: 'layers.2.attn_l', 140093222753480: 'layers.2.attn_r', 140093222753624: 'layers.2.fc.weight', 140093222754344: 'layers.2.res_fc.weight', 140093222754488: 'layers.3.attn_l', 140093222754560: 'layers.3.attn_r', 140093222754704: 'layers.3.fc.weight', 140093222755424: 'layers.3.res_fc.weight', 140093222755568: 'layers.4.attn_l', 140093222755640: 'layers.4.attn_r', 140093222755784: 'layers.4.fc.weight', 140093222744280: 'layers.4.res_fc.weight'}


'Digraph.gv.pdf'